### 数据预处理
1. 替换数据 dataframe.sex.replace({'F':1, 'M':0})
2. 

In [ ]:
c_data.Gender = c_data.Gender.replace({'F':1,'M':0})
c_data = pd.concat([c_data,pd.get_dummies(c_data['Card_Category']).drop(columns=['Platinum'])], axis=1)
c_data.drop(columns = ['Education_Level','Income_Category','Marital_Status','Card_Category','CLIENTNUM'], inplace=True)
# 展示独热编码后的特征，是不是多了几列啦

### 类别型 特征处理
1. Label encoding (sklearn LabelEncoder / pandas factorize)
2. 序列编码（Ordinal Encoding）(.map(dict))
3. 独热编码(One-Hot Encoding)
4. 频数编码（Frequency Encoding/Count Encoding) (groupby()[].agg([(,),(,)]))
5. 目标编码（Target Encoding/Mean Encoding）---
6. Beta Target Encoding
7. M-Estimate Encoding
8. James-Stein Encoding
9. Weight of Evidence Encoder
10. Leave-one-out Encoder (LOO or LOOE)
11. Binary Encoding
12. Hashing Encoding
13. Probability Ratio Encoding
14. Sum Encoder (Deviation Encoder, Effect Encoder)
15. Helmert Encoding
16. CatBoost Encoding

In [6]:
# 1. label encoding  
# 一列变一列，但不对应顺序关系
# fit transform inverse_transfrom classes_
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
city_list = ["paris", "paris", "tokyo", "amsterdam"]
le.fit(city_list)
print(le.classes_) # 查看输出种类 ['amsterdam', 'paris', 'tokyo']
city_list_en = le.transform(city_list) # encode
le.inverse_transform(city_list_en) # decode

['amsterdam' 'paris' 'tokyo']


array(['paris', 'paris', 'tokyo', 'amsterdam'], dtype='<U9')

In [15]:
# example use sklearn label
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df = pd.DataFrame({
    'pets': ['cat', 'dog', 'cat', 'monkey', 'dog', 'dog'],
    'owner': ['Champ', 'Ron', 'Brick', 'Champ', 'Veronica', 'Ron'],
    'location': ['San_Diego', 'New_York', 'New_York', 'San_Diego', 'San_Diego',
                 'New_York']
})
d = {}
le = LabelEncoder()
cols_to_encode = ['pets', 'owner', 'location']
for col in cols_to_encode:
    df[col] = le.fit_transform(df[col])
    d[col] = le.classes_

In [27]:
# label encode: use factorize of pandas
import numpy as np
import pandas as pd
df = pd.DataFrame(['green','bule','red','bule','green'],columns=['color'])
pd.factorize(df['color'])
pd.factorize(df['color'])[0]
pd.factorize(df['color'])[1]

Index(['green', 'bule', 'red'], dtype='object')

In [87]:
# Ordinal Encoding 序列编码
df = pd.DataFrame(['green','bule','red','bule','green'],columns=['color'])
ord_map = {'green':10, 'bule':33, 'red':100}
df['color'] = df['color'].map(ord_map)

In [ ]:
# One-Hot Encoding
# 高离散特征，比如工作 使用独热编码更合理
# One-Hot + PCA
from sklearn.preprocessing import OneHotEncoder
import numpy as np
df = pd.DataFrame([
    ['green', 'Chevrolet', 2017],
    ['blue', 'BMW', 2015],
    ['yellow', 'Lexus', 2018],
])
df.columns = ['color', 'make', 'year']
def LabelOneHotEncoder(data, categorical_features):
    d_num = np.array([])
    for f in data.columns:
        if f in categorical_features:
            le, ohe = LabelEncoder(), OneHotEncoder()
            data[f] = le.fit_transform(data[f])
            if len(d_num) == 0:
                d_num = np.array(ohe.fit_transform(data[[f]]))
            else:
                d_num = np.hstack((d_num, ohe.fit_transform(data[[f]]).A))
        else:
            if len(d_num) == 0:
                d_num = np.array(data[[f]])
            else:
                d_num = np.hstack((d_num, data[[f]]))
    return d_num
df_new = LabelOneHotEncoder(df, ['color', 'make', 'year'])
df_new

def one_hot_encoding(dataframe, column, train_flag=True):
    """column -> columns"""
    x = dataframe[column].values.reshape(-1, 1)
    if train_flag is True:
        ohe = OneHotEncoder().fit(x)   
        joblib.dump(ohe, 'tf_ohe.pkl')
    else:
        ohe = joblib.load('tf_ohe.pkl')
    extend_col = pd.DataFrame(ohe.transform(X).toarray())
    dataframe[ohe.get_feature_names()] = extend_col
    dataframe.drop([column], axis=1, inplace=True)
    return dataframe

In [100]:
# Frequency Encoding
data_count = df.groupby('color')['color'].agg([('cnt','size')]).reset_index()
pd.merge(df, data_count, on = 'color', how = 'left')

,color,cnt
0,10,2
1,33,2
2,100,1
3,33,2
4,10,2


In [ ]:
# 目标编码
# 高基数定性特征的例子：IP地址、电子邮件域名、城市名、家庭住址、街道、产品号码。